In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import functions as fx

from matplotlib import rc
from matplotlib import rcParams
#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Arial']

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

pad = 12
rcParams['xtick.major.pad'] = pad
rcParams['ytick.major.pad'] = pad

dpi = 200

%matplotlib inline